## Installling dependencies

In [ ]:
!pip install git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-kc5i2wgi
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-kc5i2wgi
  Resolved https://github.com/huggingface/transformers.git to commit 4a564490e149620282b9e946027e851f5f93aeae
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.4 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.32.0.dev0-py3-none-any.whl size=7447894 sha256=7e4fdac165cfd468a9fa6fb1dd6775549b74ed59439971bc42f6d93e12e20fcd
  Stored in directory: /tmp/pip-ephem-wheel-cache-1tjqxqe8/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16


In [ ]:
pip install --upgrade langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.1 MB/s eta 0:00:00


In [ ]:
!pip install -U transformers
!pip install sentencepiece
!pip install accelerate
!pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 7.3 MB/s eta 0:00:00


## Initialization of model

In [ ]:
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
import torch

In [ ]:
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(llm_int4_enable_fp32_cpu_offload=True)

tokenizer = LlamaTokenizer.from_pretrained("bigscience/bloom")

base_model = LlamaForCausalLM.from_pretrained(
    "bigscience/bloom",
    load_in_4bit=True,
    quantization_config=quantization_config,
    offload_folder="offload",
    torch_dtype=torch.float32,
    device_map='auto',
)

base_model.tie_weights()

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BloomTokenizerFast'. 
The class this function is called from is 'LlamaTokenizer'.
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565, and set the legacy attribute accordingly.


TypeError: ignored

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

model = "tiiuae/falcon-40b"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)
sequences = pipeline(
   "Girafatron is obsessed with giraffes, the most glorious animal on the face of this Earth. Giraftron believes all other animals are irrelevant when compared to the glorious majesty of the giraffe.\nDaniel: Hello, Girafatron!\nGirafatron:",
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")


ImportError: ignored

In [ ]:
pipe = pipeline(
    "text-generation",
    model=base_model,
    tokenizer=tokenizer,
    max_length=512,
    temperature=0,
    top_p=0.95,
    repetition_penalty=1.2
)

local_llm = HuggingFacePipeline(pipeline=pipe)

## Defining prompt and usage

In [ ]:
from langchain import PromptTemplate, LLMChain

template = """Below is an instruction that describes a certain task. Write a long response that fully completes the given request.


### Instruction:
{instruction}

Answer:"""

prompt = PromptTemplate(template=template, input_variables=["instruction"])

In [ ]:
from langchain import PromptTemplate, LLMChain

prompt2 = """
Please create me a tweet about goin to the park and eating a sandwich.

% Tone
 -Don't use any hashtags.
 -Respond in the tone of Bill Gates
"""

In [ ]:
llm_chain = LLMChain(prompt=prompt,
                     llm=local_llm
                     )

question = "Hey i want to learn german"

print(llm_chain.run(question))
print(llm_chain.run(prompt2))

 I'm going to the park with my friends to enjoy the sunshine and eat a delicious sandwich! #takingabreak #sandwhichlife


In [ ]:
print(local_llm.predict(prompt2))

Answer: Create your own tweets! Here's an example of what you could do: "Going to the park with my friends and can't wait to eat this delicious sandwich I made!"
